# RefLex LLM - OpenAI Integration Example

This notebook demonstrates how to use RefLex LLM specifically with OpenAI endpoints, including configuration, fallback capabilities, and best practices.

## Installation

In [1]:
!pip install reflex-llms


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Basic OpenAI Usage

In [8]:
from reflex_llms import (
    get_openai_client, 
    get_selected_provider,
)

# Get client with OpenAI preference
client = get_openai_client(
    preference_order=["openai", "reflex"],
    openai_base_url="https://wrong.address.com/v1",
)

# Use exactly like the OpenAI client
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello! I'm using OpenAI through RefLex."}
    ],
    max_tokens=100
)

print(f"Response: {response.choices[0].message.content}")
print(f"Using provider: {get_selected_provider()}")

Using cached reflex configuration
Response: OpenAI and Reflex are innovative tools for natural language understanding and generation. How can I assist you today? Are you working on a specific project or facing any challenges with the technology? I'm here to help you find answers or provide guidance.
Using provider: reflex


In [14]:
client = get_openai_client(
    preference_order=["openai", "reflex"],
    openai_base_url="https://wrong.address.com/v1",
)

Using cached reflex configuration


In [5]:
import reflex_llms

# Use exactly like the OpenAI client
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input="Hello! I'm using OpenAI through RefLex."
)

print(f"Response: {response}")
print(f"Using provider: {reflex_llms.get_selected_provider()}")

Response: CreateEmbeddingResponse(data=[Embedding(embedding=[0.0008691217, 0.027443694, -0.1357695, -0.006981929, 0.043804947, -0.066955015, 0.06408625, -0.007891853, 0.011743882, -0.03970715, 0.028807608, 0.06722247, 0.05095649, 0.01118963, -0.0049538044, -0.018620186, 0.06305547, -0.011259739, -0.021611279, 0.010976157, -0.0043819984, -0.041408855, -0.0062828725, 0.03632305, 0.15982847, 0.03036786, 0.0431361, 0.043358844, -0.08415229, -0.031661786, 0.06642479, -0.04764157, 0.013351845, 0.015142062, -0.029296294, 0.0067227646, -0.021725835, 0.0426883, -0.0365096, 0.03347313, 0.0087062735, 0.010531868, 0.021179292, 0.0046447953, 0.0796105, -0.032350607, -0.0017474409, 0.0009717375, 0.07587133, -0.025479928, 0.019269366, -0.017003775, -0.09584733, -0.027847186, 0.05083585, 0.0024121522, -0.015586027, 0.034156267, 0.02212853, -0.05065058, 0.033891913, 0.027409155, -0.022231242, 0.043184325, 0.045827392, 0.025870772, -0.003041088, 0.038481493, 0.035482824, -0.02303227, 0.0115332175, 0.017

In [6]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello! I'm using OpenAI through RefLex."}],
    max_tokens=200,
    stream=True
)

full_response = ""
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        content = chunk.choices[0].delta.content
        print(content, end="", flush=True)
        full_response += content

Hi there! It's nice to meet you. Are you getting help with a project or have a specific question about using the OpenAI model through Reflex? I'd be happy to try and assist you. What can I help you with today?

In [17]:
stream = client.chat.completions.create(
    model="o4-mini",
    messages=[{"role": "user", "content": "Hello! I'm using OpenAI through RefLex."}],
    max_tokens=200,
    stream=True
)

NotFoundError: Error code: 404 - {'error': {'message': 'model "o4-mini" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}

In [9]:
from reflex_llms import (
    get_reflex_server, 
    is_using_reflex,
)

client2 = get_openai_client()
server = get_reflex_server()

Using cached reflex configuration


In [ ]:
if server:
    print(f"RefLex server available: {server.openai_compatible_url}")
    print(f"Server healthy: {server.is_healthy}")
else:
    print("Not using RefLex server")

print(f"\nSelected provider: {get_selected_provider()}")
print(f"Using RefLex: {is_using_reflex()}")